# Streamlit 실습

In [1]:
!pip install streamlit openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
%%writefile chatbot_app.py

import streamlit as st
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()


with st.sidebar:
    openai_api_key = st.text_input("OpenAI API Key", key="chatbot_api_key", type="password")
    "[Get an OpenAI API](https://platform.openai.com/account/api-keys)"

st.title("Chat bot")

if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]

# 챗 내용
for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])


if prompt := st.chat_input():
    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    
    client = OpenAI(api_key=openai_api_key)
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=st.session_state.messages)
    msg = response.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": msg})
    st.chat_message("assistant").write(msg)
    


Writing chatbot_app.py


In [5]:
#!streamlit run ./chatbot_app.py &> /logs.txt &
#!npx localtunnel --port 8501 & curl ipv4.icanhazip.com


# local 터미널 실행
streamlit run chatbot_app.py

SyntaxError: invalid syntax (453986283.py, line 6)

# 참고 : Anthropic 이용 방법
- 문서를 요약해주는 예제

In [ ]:
!pip install anthropic

In [ ]:
%%writefile anthropic_qna_bot.py

import streamlit as st
import anthropic

with st.sidebar:
    anthropic_api_key = st.text_input("Anthropic API KEy", key="file_qa_api_key", type="password")

    st.title("File Q&A with Anthropic")

    uploaded_file = st.file_uploader("Upload an article", type=("txt", "md"))

    question = st.text_input(
        "Ask something about the article",
        placeholder="Can you give me a short summary?",
        disabled=not uploaded_file,
    )

    if uploaded_file and question and not anthropic_api_key:
        st.info("Please add your Anthropic API key to continue.")

    if upload_file and question and anthropic_api_key:
        article = uploaded_file.read().decode()
        prompt = f"""{anthropic.HUMAN_PROMPT} Here's an article:\n\n
        {article}\n\n\n\n{question}{anthropic.AI_PROMPT}
        """

        client = anthropic.Client(api_key=anthropic_api_key)
        response = client.completions.creeate(
            prompt=prompt,
            stop_sequences=[anthropic.HUMAN_PROMPT],
            model="claude-v1",
            max_tokens_to_sample=100,
        )

        st.write("### Answer")
        st.write(response.completion)

In [ ]:
!streamlit run /content/anthropic_qna_bot.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

# Search With Langchain

In [7]:
!pip install langchain langchain-community duckduckgo-search 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.5 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
%%writefile search_langchain_app.py
import streamlit as st

from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

with st.sidebar:
    openai_api_key = st.text_input("OpenAI API Key", key="langchain_search_api_key_openai", type="password")
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/pages/2_Chat_with_search.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("🔎 LangChain - Chat with search")

"""
In this example, we're using `StreamlitCallbackHandler` to display the thoughts and actions of an agent in an interactive Streamlit app.
Try more LangChain 🤝 Streamlit Agent examples at [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).
"""

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "assistant", "content": "Hi, I'm a chatbot who can search the web. How can I help you?"}
    ]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input(placeholder="Who won the Women's U.S. Open in 2018?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, streaming=True)
    #search = DuckDuckGoSearchRun(name="Search")
    search = DuckDuckGoSearchRun()
    search_agent = initialize_agent([search], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)
    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container(), expand_new_thoughts=False)
        response = search_agent.run(st.session_state.messages, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)

Writing search_langchain_app.py


In [ ]:
#!streamlit run ./search_langchain_app.py &> /logs.txt &
#!npx localtunnel --port 8501 & curl ipv4.icanhazip.com


# local 터미널 실행
streamlit run search_langchain_app.py